In [1]:
import pandas as pd

In [2]:
teams_df = pd.read_excel('train.xlsx')
current_df = pd.read_excel('test.xlsx')

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
teams_df = teams_df.drop(columns=['Date'])
current_df = current_df.drop(columns=['Date'])

In [5]:
X = teams_df.drop(columns=['Target'])
y = teams_df['Target']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

X_test = current_df.drop(columns=['Target'])
y_test = current_df['Target']

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [8]:
from sklearn.model_selection import GridSearchCV

In [9]:
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4],
    'class_weight': [None, 'balanced']
}

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [11]:
grid_search = GridSearchCV(estimator=SVC(), param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'class_weight': [None, 'balanced'],
                         'degree': [2, 3, 4], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf']},
             scoring='accuracy')

In [12]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'C': 0.1, 'class_weight': None, 'degree': 2, 'gamma': 'auto', 'kernel': 'rbf'}
Best Score: 0.6043606840992463


In [13]:
best_svm_classifier = SVC(**best_params)
best_svm_classifier.fit(X_train, y_train)

SVC(C=0.1, degree=2, gamma='auto')

In [14]:
y_val_pred = best_svm_classifier.predict(X_val)
print('Report:\n', classification_report(y_val, y_val_pred))
print('Confusion Matrix:\n', confusion_matrix(y_val, y_val_pred))

Report:
               precision    recall  f1-score   support

           0       0.62      0.63      0.62       912
           1       0.62      0.61      0.61       912

    accuracy                           0.62      1824
   macro avg       0.62      0.62      0.62      1824
weighted avg       0.62      0.62      0.62      1824

Confusion Matrix:
 [[575 337]
 [359 553]]


In [15]:
y_test_pred = best_svm_classifier.predict(X_test)
print('Report:\n', classification_report(y_test, y_test_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_test_pred))

Report:
               precision    recall  f1-score   support

           0       0.67      0.58      0.62       805
           1       0.63      0.71      0.67       805

    accuracy                           0.65      1610
   macro avg       0.65      0.65      0.64      1610
weighted avg       0.65      0.65      0.64      1610

Confusion Matrix:
 [[464 341]
 [230 575]]


In [16]:
import joblib

In [17]:
file = 'TrainedModels/SVM_Model.pkl'
scaler_file = 'Scalers/SVM.bin'
joblib.dump(best_svm_classifier, file)
joblib.dump(scaler, scaler_file, compress=True)

['Scalers/SVM.bin']

In [18]:
svm_model = joblib.load(file)

In [19]:
y_test_pred = svm_model.predict(X_test)
print('Report:\n', classification_report(y_test, y_test_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_test_pred))

Report:
               precision    recall  f1-score   support

           0       0.67      0.58      0.62       805
           1       0.63      0.71      0.67       805

    accuracy                           0.65      1610
   macro avg       0.65      0.65      0.64      1610
weighted avg       0.65      0.65      0.64      1610

Confusion Matrix:
 [[464 341]
 [230 575]]
